In [2]:
import pandas as pd
import numpy as np
import json

In [7]:
filepath = "../TO_OrgCharts.json"

# Opening JSON file
f = open(filepath)
  
# Returns JSON object as a dictionary
data = json.load(f)
# Iterating through the json list
# for i in data['emp_details']:
#     print(i)
  
# Closing file


In [14]:
print(data)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Make Companies CSV

In [66]:
cols = ['CompanyName', 'twitterUrl', 'linkedInUrl', 'facebookUrl', 'websiteUrl', \
        'city', 'country', 'countryIso', 'state', 'locationString', 'isPrimary', \
        'employeeSizeRange', 'description']
company_df_dict = {colname: [] for colname in cols}

for line in data['orgCharts']:
    company_df_dict['CompanyName'].append(line['CompanyName'])

    company_df_dict['twitterUrl'].append(line['stats']['social']['twitterUrl'])
    company_df_dict['linkedInUrl'].append(line['stats']['social']['linkedInUrl'])
    company_df_dict['facebookUrl'].append(line['stats']['social']['facebookUrl'])
    company_df_dict['websiteUrl'].append(line['stats']['social']['websiteUrl'])
    
    company_df_dict['city'].append(line['stats']['location']['city'])
    company_df_dict['country'].append(line['stats']['location']['country'])
    company_df_dict['countryIso'].append(line['stats']['location']['countryIso'])
    company_df_dict['state'].append(line['stats']['location']['state'])
    company_df_dict['locationString'].append(line['stats']['location']['locationString'])
    company_df_dict['isPrimary'].append(line['stats']['location']['isPrimary'])
    
    company_df_dict['employeeSizeRange'].append(line['stats']['employeeRange'])
    
    company_df_dict['description'].append(line['stats']['description'])

company_df = pd.DataFrame.from_dict(company_df_dict)

In [67]:
company_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12171 entries, 0 to 12170
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   CompanyName        12171 non-null  object
 1   twitterUrl         9957 non-null   object
 2   linkedInUrl        11615 non-null  object
 3   facebookUrl        9124 non-null   object
 4   websiteUrl         12171 non-null  object
 5   city               12171 non-null  object
 6   country            12171 non-null  object
 7   countryIso         12171 non-null  object
 8   state              3042 non-null   object
 9   locationString     10510 non-null  object
 10  isPrimary          12171 non-null  bool  
 11  employeeSizeRange  12171 non-null  object
 12  description        12171 non-null  object
dtypes: bool(1), object(12)
memory usage: 1.1+ MB


In [68]:
company_df['employeeSizeRange'].value_counts()

50-200     10225
200-500     1946
Name: employeeSizeRange, dtype: int64

In [69]:
# Mostly missing states (only "NV" appears), so try to fill in from locationString
us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
}

abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))

def fill_state_abbrev(desc):
    if desc is not None: 
        states = us_state_to_abbrev.keys()
        for state in states: 
            if state in desc: 
                abbrev = us_state_to_abbrev[state]
                return abbrev
    return None

company_df['stateFilled'] = company_df['locationString'].apply(lambda x: fill_state_abbrev(x))
company_df['stateAbbrev'] = company_df.apply(lambda row: row["state"] if row["state"] is not None else row["stateFilled"], axis=1)
company_df['stateName'] = company_df['stateAbbrev'].apply(lambda x: abbrev_to_us_state[x])
company_df.drop(columns=['stateFilled'], inplace=True)

# Fill in based on cleaned city/state/country columns, some location strings missing
company_df['locationString'] = company_df.apply(lambda row: row["city"] + ", " + row["stateName"] + ", " + row["country"], axis=1)

# Save
company_df.to_csv("company_df.csv")

In [70]:
# Close file
f.close()

## Make Org Chart CSV

In [106]:
cols = ['CompanyName', 'id', 'title', 'leafMember', 'containingNodeId', 'order', 'parentId', 'section', 'type']
orgchart_df_dict = {colname: [] for colname in cols}

for line in data['orgCharts']:
    employeeNodes = line['employeeNodes']
    for i in range(len(employeeNodes)):
        if line['employeeNodes'][i]['__typename'] == 'OrgChartStructureNode': 
            
            for col in cols: 
                if col == 'CompanyName': 
                    orgchart_df_dict['CompanyName'].append(line['CompanyName'])
                else: 
                    orgchart_df_dict[col].append(line['employeeNodes'][i][col])
        # Print type if not OrgChartStructureNode 
        else: 
            print(line['employeeNodes'][i]['__typename'])
            
orgchart_df = pd.DataFrame.from_dict(orgchart_df_dict)
orgchart_df

,CompanyName,id,title,leafMember,containingNodeId,order,parentId,section,type
0,Oltmans Construction Co.,p-1517136,Charles Roy,"{'__typename': 'FlatPosition', 'id': 1517136, ...",None,1,p-1517109,orgChart,leaf
1,Oltmans Construction Co.,p-1517160,Jason Kakimoto,"{'__typename': 'FlatPosition', 'id': 1517160, ...",None,0,p-1517109,orgChart,leaf
2,Oltmans Construction Co.,p-1517254,John Dang,"{'__typename': 'FlatPosition', 'id': 1517254, ...",None,0,p-1517139,orgChart,leaf
3,Oltmans Construction Co.,p-1517248,Louise Le,"{'__typename': 'FlatPosition', 'id': 1517248, ...",None,17,p-1517109,orgChart,leaf
4,Oltmans Construction Co.,p-1517247,Brian Irvin,"{'__typename': 'FlatPosition', 'id': 1517247, ...",None,16,p-1517109,orgChart,leaf
...,...,...,...,...,...,...,...,...,...
336066,Windes,p-219447,Jeffrey S. Fields,"{'__typename': 'FlatPosition', 'id': 219447, '...",None,-1,p-217320,orgChart,leaf
336067,Windes,p-219436,Christy E. Woods,"{'__typename': 'FlatPosition', 'id': 219436, '...",None,-1,p-217320,orgChart,leaf
336068,Windes,p-217320,"Guy A. Nicio, CPA, MST","{'__typename': 'FlatPosition', 'id': 217320, '...",None,9,p-217287,orgChart,leaf
336069,Windes,p-218127,John Di Carlo,"{'__typename': 'FlatPosition', 'id': 218127, '...",None,1,None,board,leaf


In [107]:
# Clean id's to not include "p-"
orgchart_df["id"] = orgchart_df["id"].apply(lambda x: None if x is None else int(x[2:]) if x[0:2] == 'p-' else x)
orgchart_df["parentId"] = orgchart_df["parentId"].apply(lambda x: None if x is None else int(x[2:]) if x[0:2] == 'p-' else x)

# Rename title to name
orgchart_df.rename(columns={"id": "employeeId", "title": "employeeName"}, inplace=True)

In [108]:
orgchart_df

,CompanyName,employeeId,employeeName,leafMember,containingNodeId,order,parentId,section,type
0,Oltmans Construction Co.,1517136,Charles Roy,"{'__typename': 'FlatPosition', 'id': 1517136, ...",None,1,1517109,orgChart,leaf
1,Oltmans Construction Co.,1517160,Jason Kakimoto,"{'__typename': 'FlatPosition', 'id': 1517160, ...",None,0,1517109,orgChart,leaf
2,Oltmans Construction Co.,1517254,John Dang,"{'__typename': 'FlatPosition', 'id': 1517254, ...",None,0,1517139,orgChart,leaf
3,Oltmans Construction Co.,1517248,Louise Le,"{'__typename': 'FlatPosition', 'id': 1517248, ...",None,17,1517109,orgChart,leaf
4,Oltmans Construction Co.,1517247,Brian Irvin,"{'__typename': 'FlatPosition', 'id': 1517247, ...",None,16,1517109,orgChart,leaf
...,...,...,...,...,...,...,...,...,...
336066,Windes,219447,Jeffrey S. Fields,"{'__typename': 'FlatPosition', 'id': 219447, '...",None,-1,217320,orgChart,leaf
336067,Windes,219436,Christy E. Woods,"{'__typename': 'FlatPosition', 'id': 219436, '...",None,-1,217320,orgChart,leaf
336068,Windes,217320,"Guy A. Nicio, CPA, MST","{'__typename': 'FlatPosition', 'id': 217320, '...",None,9,217287,orgChart,leaf
336069,Windes,218127,John Di Carlo,"{'__typename': 'FlatPosition', 'id': 218127, '...",None,1,None,board,leaf


In [109]:
orgchart_df.iloc[336068]["leafMember"]

{'__typename': 'FlatPosition',
 'id': 217320,
 'slug': 'guy-a-nicio-cpa-mst',
 'fullName': 'Guy A. Nicio, CPA, MST',
 'role': 'Partner - Chairman, Tax & Accounting Services',
 'roleFunction': None,
 'roleAutoFunction': 'admn',
 'description': '',
 'parentPositionId': 217287,
 'profileImage': {'__typename': 'Image',
  'endpoint': 'https://cdn.theorg.com',
  'ext': 'jpg',
  'placeholderDataUrl': None,
  'prevailingColor': 'a28a82',
  'uri': '639dc980-4a62-4935-97cd-b832aad54511',
  'versions': ['thumb', 'small', 'medium', 'large']},
 'social': {'__typename': 'UserSocial',
  'twitterUrl': 'https://twitter.com/Windes1926',
  'linkedInUrl': 'http://www.linkedin.com/company/48010',
  'facebookUrl': 'https://www.facebook.com/windesinc/',
  'websiteUrl': 'https://windes.com/'},
 'isAdviser': False,
 'group': None,
 'companyStartDate': None,
 'roleStartDate': None,
 'location': None,
 'invitedAt': None,
 'remote': False,
 'lastUpdate': '2022-08-24T15:16:48.624',
 'pronoun': None,
 'claimedBy': 

In [110]:
cols = ["slug", "role", "roleFunction", "roleAutoFunction", "description", "parentPositionId", "isAdviser", "group", 
       "companyStartDate", "roleStartDate", "location", "invitedAt", "remote", "lastUpdate", "pronoun", "claimedBy"]
for colname in cols:
    orgchart_df[colname] = orgchart_df["leafMember"].apply(lambda leaf_dict: leaf_dict[colname] if leaf_dict is not None and colname in leaf_dict.keys() else None)
    
social_cols = ["twitterUrl", "linkedInUrl", "facebookUrl", "websiteUrl"]
for colname in social_cols:
    orgchart_df[colname] = orgchart_df["leafMember"].apply(lambda leaf_dict: leaf_dict['social'][colname] if leaf_dict is not None and colname in leaf_dict.keys() else None)

cols_to_drop = ['leafMember', 'containingNodeId', 'roleFunction', 'group', 'companyStartDate', 'roleStartDate', 
               'location', 'invitedAt', 'pronoun', 'claimedBy', 'section'] + social_cols
orgchart_df.drop(columns=cols_to_drop, inplace=True)

In [111]:
orgchart_df.to_csv("orgchart_df.csv")

In [112]:
orgchart_df

,CompanyName,employeeId,employeeName,order,parentId,section,type,slug,role,roleAutoFunction,description,parentPositionId,isAdviser,remote,lastUpdate
0,Oltmans Construction Co.,1517136,Charles Roy,1,1517109,orgChart,leaf,charles-roy,President,admn,With Oltmans Since 1984 University of Californ...,1517109.0,False,False,2022-07-11T13:30:04.569
1,Oltmans Construction Co.,1517160,Jason Kakimoto,0,1517109,orgChart,leaf,jason-kakimoto,Vice President & CFO,finc,Jason Kakimoto was named vice president and me...,1517109.0,False,False,2022-07-11T13:42:56.943
2,Oltmans Construction Co.,1517254,John Dang,0,1517139,orgChart,leaf,john-dang,"Associate Director, Business Development",bd,As a leader in Oltmans’ client relations and b...,1517139.0,False,False,2022-07-11T13:41:20.440
3,Oltmans Construction Co.,1517248,Louise Le,17,1517109,orgChart,leaf,louise-le,"Director, Marketing",mktg,Joining the team as Oltmans Construction Co.’s...,1517109.0,False,False,2022-07-11T13:40:37.563
4,Oltmans Construction Co.,1517247,Brian Irvin,16,1517109,orgChart,leaf,brian-irvin,"Director, Information Technology",it,"As director of IT, Brian oversees an in-house ...",1517109.0,False,False,2022-07-11T13:40:23.796
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336066,Windes,219447,Jeffrey S. Fields,-1,217320,orgChart,leaf,jeffrey-s-fields,"Partner-in-Charge, Tax & Accounting Services -...",acct,Jeff joined Windes in 2011 and is a partner in...,217320.0,False,False,2022-08-31T15:54:55.413
336067,Windes,219436,Christy E. Woods,-1,217320,orgChart,leaf,christy-e-woods,"Partner-in-Charge, Tax & Accounting Services -...",acct,Christy is a Partner in the firm’s Tax & Accou...,217320.0,False,False,2022-08-31T16:01:37.249
336068,Windes,217320,"Guy A. Nicio, CPA, MST",9,217287,orgChart,leaf,guy-a-nicio-cpa-mst,"Partner - Chairman, Tax & Accounting Services",admn,,217287.0,False,False,2022-08-24T15:16:48.624
336069,Windes,218127,John Di Carlo,1,None,board,leaf,john-di-carlo,Board Member,admn,John joined [Windes](/org/windes) in 1981 and ...,NaN,True,False,2022-08-31T15:59:59.398
